# Bibliography Categorization: 'BibCat'
## Tutorial: Machine learning (ML) models in bibcat.



---


## Introduction.

In this tutorial, we will use bibcat to train a machine learning (ML) model on some raw input text.


---

## User Workflow: Training a machine learning (ML) model.


The `Operator` class contains a user-friendly method `train_model_ML` that runs the full workflow for training an ML model, from the input raw text all the way to saving the output ML model.  We overview how this method can be run in the code blocks below.

For this tutorial, we have two sets of data: either 1) some short, made-up text for a quick run of the code, or 2) an imported database of text from an external file of the user's choosing. The former case is useful for getting a quick sense of how the code works. The latter case is useful for building an actual model, but of course will take much longer on larger databases of text!

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Import external packages
import os
import time
import json

In [3]:
# Set up for fetching necessary bibcat modules for the tutorial
# Check work directories: src/ is where all source python scripts are available. 
current_dir= os.path.dirname(os.path.abspath('__file__'))
_parent = os.path.dirname(current_dir)
src_dir = os.path.join(_parent, "src")

print(f'Current Directory: {current_dir}')
print(f'Source directory: {src_dir}')

# move to the ../src/ directory to import necessary modules. 
os.chdir(src_dir)

Current Directory: /Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/docs
Source directory: /Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/src


In [4]:
#Import bibcat packages
import bibcat_classes as bibcat
import bibcat_config as config
import bibcat_constants as preset

Root directory =/Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/src, parent directory=/Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat
/Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/scratchwork/models folder already exists.
/Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/scratchwork/output folder already exists.


In [5]:
#Which data would you like to run the ML model on?  Choose from the booleans below.
do_quick_run = False #This will train the ML model on short bits of text. Runs pretty quickly.
do_real_run = True #This will train the ML model on external text. Will take longer for larger databases.
#

In [6]:
#The rest of these parameters can be left as-is for a first run-through.
#
num_papers = 500 #500 #None, or an integer; if an integer, will truncate external .json text dataset to this size
#Set num_papers=None to use all available papers in external dataset
#Note: If set to integer, final paper count might be a little more than target num_papers given
#
allowed_classifications = config.allowed_classifications #For external data; classifications to include
#

#Fetch filepaths for model and data
name_model = config.name_model
filepath_json = config.path_json
dir_model = os.path.join(config.dir_allmodels, name_model)
#
#Set values for generating ML model
do_reuse_run = True #Whether or not to reuse any existing output from previous training+validation+testing runs
do_shuffle = True #Whether or not to shuffle contents of training vs validation vs testing datasets
fraction_TVT = [0.8, 0.1, 0.1] #Fractional breakdown of training vs validation vs testing datasets
#
mode_TVT = "uniform" # "uniform" #"available"
#"uniform" = all training datasets will have the same number of entries
#"available" = all training datasets will use full fraction (from fraction_TVT) of data available
#
seed_TVT = 10 #Random seed for generating training vs validation vs testing datasets
seed_ML = 8 #Random seed for ML model
mode_modif = "skim_trim_anon" #Mode to use for processing and generating modifs from input raw text
#NOTE: See other modif modes in workflow tutorial
buffer = 0
#
#Prepare some Keyword objects
kobj_hubble = bibcat.Keyword(
                keywords=["Hubble", "Hubble Telescope",
                          "Hubble Space Telescope"],
                acronyms=["hst", "ht"])
all_kobjs = [kobj_hubble]
#

In [7]:
#Initialize an empty ML classifier
classifier_ML = bibcat.Classifier_ML(filepath_model=None, fileloc_ML=None,
                                    class_names=None, do_verbose=True)
#

In [8]:
#Initialize an Operator
tabby_ML = bibcat.Operator(classifier=classifier_ML, mode=mode_modif, keyword_objs=all_kobjs,
                           do_verbose=True, load_check_truematch=False, do_verbose_deep=False)
#

Instance of Operator successfully initialized!
Keyword objects:
0: Keyword Object:
Name: Hubble
Keywords: ['Hubble Space Telescope', 'Hubble Telescope', 'Hubble']
Acronyms: ['hst', 'ht']



In [9]:
#Set up data for the quick example case. But in reality, ML models should be trained on MUCH more data than this!!!
if do_quick_run:
    #Make some fake data
    dict_texts_raw = {"science":["We present HST observations in Figure 4.",
                            "The HST stars are listed in Table 3b.",
                            "Despite our efforts to smooth the data, there are still rings in the HST images.",
                            "See Section 8c for more discussion of the Hubble images.",
                            "The supernovae detected with HST tend to be brighter than initially predicted.",
                            "Our spectra from HST fit well to the standard trend first published in Someone et al. 1990.",
                            "We use the Hubble Space Telescope to build an ultraviolet database of the target stars.",
                            "The blue points (HST) exhibit more scatter than the red points (JWST).",
                            "The benefit, then, is the far higher S/N we achieved in our HST observations.",
                            "Here we employ the Hubble Telescope to observe the edge of the photon-dominated region.",
                            "The black line shows that the region targeted with Hubble has an extreme UV signature."],
                     "datainfluenced":["The simulated Hubble data is plotted in Figure 4.",
                           "Compared to the HST observations in Someone et al., our JWST follow-up reached higher S/N.",
                           "We were able to reproduce the luminosities from Hubble using our latest models.",
                           "We overplot Hubble-observed stars from Someone et al. in Figure 3b.",
                           "We built the spectral templates using UV data in the Hubble archive.",
                           "We simulate what our future HST observations will look like to predict the S/N.",
                           "Our work here with JWST is inspired by our earlier HST study published in 2010.",
                           "We therefore use the Hubble statistics from Author et al. to guide our stellar predictions.",
                           "The stars in Figure 3 were plotted based on the HST-fitted trend line in Person et al.",
                           "The final step is to use the HST exposure tool to put our modeled images in context."],
                     "mention":["Person et al. used HST to measure the Hubble constant.",
                            "We will present new HST observations in a future work.",
                            "HST is do_a fantastic instrument that has revolutionized our view of space.",
                            "The Hubble Space Telescope (HST) has its mission center at the STScI.",
                            "We can use HST to power a variety of science in the ultraviolet regime.",
                            "It is not clear when the star will be observable with HST.",
                            "More data can be found and downloaded from the Hubble archive.",
                            "We note that HST can be used to observe the stars as well, at higher S/N.",
                            "However, we ended up using the JWST rather than HST observations in this work.",
                            "We push the analysis of the Hubble component of the dataset to a future study.",
                            "We expect the HST observations to be released in the fall.",
                            "We look forward to any follow-up studies with, e.g., the Hubble Telescope."]}
    #
    #Convert into dictionary with: key:text,class,id,mission structure
    i_track = 0
    dict_texts = {}
    for key in dict_texts_raw:
        curr_set = dict_texts_raw[key]
        for ii in range(0, len(curr_set)):
            dict_texts[str(i_track)] = {"text":curr_set[ii], "class":key, "id":"{0}_{1}".format(key, ii),
                                       "mission":"HST"}
            i_track += 1
#

In [10]:
#Set up data for the external data case.
if do_real_run:
    #Load the original data
    with open(filepath_json, 'r') as openfile:
        dataset = json.load(openfile)
        len(dataset)
    #
    #Organize a new version of the data with: key:text,class,id,mission structure
    i_track = 0 #Track number of papers kept from original dataset
    dict_texts = {}
    for ii in range(0, len(dataset)):
        #Extract mission classifications for current text
        curr_data = dataset[ii]
        #
        #Skip if no valid text at all for this text
        if ("body" not in curr_data):
            continue
        #
        #Skip if no valid missions at all for this text
        if ("class_missions" not in curr_data):
            continue
        #
        #Otherwise, extract the missions
        curr_missions = curr_data["class_missions"]
        #
        print(curr_missions)
        #Iterate through missions for this text
        i_mission = 0
        for curr_key in curr_missions:
            #If this is not an allowed mission, skip
            if (curr_missions[curr_key]["papertype"] not in allowed_classifications):
                continue
            #
            #Otherwise, check if this mission is a target mission
            fetched_kobj = tabby_ML._fetch_keyword_object(lookup=curr_key,
                                                          do_verbose=False, do_raise_emptyerror=False)
            #Skip if not a target
            if (fetched_kobj is None):
                continue
            #
            #Otherwise, store classification info for this entry
            curr_class = curr_missions[curr_key]["papertype"]
            new_dict = {"text":curr_data["body"], #Text for this paper
                        "class":curr_class, #Classification for this mission
                        "mission":curr_key, #The mission itself
                        "id":("paper{0}_mission{1}_{2}_{3}".format(ii, i_mission,
                                                                   curr_key, curr_class)) #ID for this entry
                       }
            dict_texts[str(i_track)] = new_dict
            #
            #Increment counters
            i_mission += 1 #Count of kept missions for this paper
            i_track += 1 #Count of kept classifications overall
        #

        #Terminate early if requested number of papers reached
        if ((num_papers is not None) and (i_track >= num_papers)):
            break
    #
#

19782

{'': {'bibcode': '2021MNRAS.500.3002B', 'papertype': '', 'is_falsepos': True}}
{'Roman': {'bibcode': '2021MNRAS.500.3254R', 'papertype': 'MENTION', 'is_falsepos': False}, 'PanSTARRS': {'bibcode': '2021MNRAS.500.3254R', 'papertype': 'DATA_INFLUENCED', 'is_falsepos': False}}
{'JWST': {'bibcode': '2020SSRv..216...50C', 'papertype': 'MENTION', 'is_falsepos': False}}
{'HST': {'bibcode': '2020SSRv..216...69A', 'papertype': 'DATA_INFLUENCED', 'is_falsepos': False}, 'JWST': {'bibcode': '2020SSRv..216...69A', 'papertype': 'MENTION', 'is_falsepos': False}, 'Roman': {'bibcode': '2020SSRv..216...69A', 'papertype': 'MENTION', 'is_falsepos': False}}
{'': {'bibcode': '2020SSRv..216...38H', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020SSRv..216...81A', 'papertype': '', 'is_falsepos': True}}
{'PanSTARRS': {'bibcode': '2021MNRAS.500.4004D', 'papertype': 'MENTION', 'is_falsepos': False}}
{'GALEX': {'bibcode': '2020SSRv..216..124V', 'papertype': 'MENTION', 'is_falsepos': False}, 'PanSTARRS

{'': {'bibcode': '2020SPIE11246E..04S', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020SPIE11236E..0LB', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020SPIE11231E..0DK', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020RAA....20...29H', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020SPIE11240E..0RX', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020SPIE11240E..0SL', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020SPIE11244E..0SR', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020SPIE11256E..0QG', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020SPIE11219E....C', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020SPIE11240E..2RL', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020SPIE11228E....I', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020SPIE11240E..10S', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020SPIE11255E..09M', 'papertype':

{'': {'bibcode': '2020MNRAS.496.4287S', 'papertype': '', 'is_falsepos': True}}
{'HST': {'bibcode': '2020NatAs...4.1130A', 'papertype': 'MENTION', 'is_falsepos': False}, 'JWST': {'bibcode': '2020NatAs...4.1130A', 'papertype': 'MENTION', 'is_falsepos': False}}
{'': {'bibcode': '2020MNRAS.499.6053L', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020MNRAS.497L..62B', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020MNRAS.497.1463B', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020P&SS..18104818L', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020P&SS..18204796T', 'papertype': '', 'is_falsepos': True}}
{'': {'bibcode': '2020MNRAS.497..976K', 'papertype': '', 'is_falsepos': True}}
{'TESS': {'bibcode': '2020MNRAS.499.2817V', 'papertype': 'MENTION', 'is_falsepos': False}, 'KEPLER': {'bibcode': '2020MNRAS.499.2817V', 'papertype': 'SCIENCE', 'is_falsepos': False}}
{'': {'bibcode': '2020Natur.577..717W', 'papertype': '', 'is_falsepos': True}}
{'HST

In [11]:
#Throw error if not enough text entries collected
if do_real_run:
    if ((num_papers is not None) and (len(dict_texts) < num_papers)):
        raise ValueError("Err: Something went wrong during initial processing. Insufficient number of texts extracted."
                        +"\nRequested number of texts: {0}\nActual number of texts: {1}"
                        .format(num_papers, len(dict_texts)))
#

In [12]:
#Uncomment the code below to print a snippet of each of the entries in the dataset.
#"""
for curr_key in dict_texts:
    print("Text #{0}:".format(curr_key))
    print("Classification: {0}".format(dict_texts[curr_key]["class"]))
    print("Mission: {0}".format(dict_texts[curr_key]["mission"]))
    print("ID: {0}".format(dict_texts[curr_key]["id"]))
    print("Text snippet:")
    print(dict_texts[curr_key]["text"][0:500])
    print("---\n\n")
#"""

Text #0:
Classification: DATA_INFLUENCED
Mission: HST
ID: paper3_mission0_HST_DATA_INFLUENCED
Text snippet:
Space Sci Rev (2020) 216:69 https://doi.org/10.1007/s11214-020-00690-x Star Clusters Near and Far Tracing Star Formation Across Cosmic Time Angela Adamo1 · Peter Zeidler2,3 · J.M. Diederik Kruijssen4 · Mélanie Chevance4 · Mark Gieles5,6 · Daniela Calzetti7 · Corinne Charbonnel8,9 · Hans Zinnecker10 · Martin G.H. Krause11 Received: 31 January 2020 / Accepted: 11 May 2020 / Published online: 8 June 2020 © The Author(s) 2020 Abstract Star clusters are fundamental units of stellar feedback and uniqu
---


Text #1:
Classification: MENTION
Mission: HST
ID: paper7_mission0_HST_MENTION
Text snippet:
Space Sci Rev (2020) 216:124 https://doi.org/10.1007/s11214-020-00753-z Optical-Ultraviolet Tidal Disruption Events Sjoert van Velzen1,2 · Thomas W.-S. Holoien3 · Francesca Onori4 · Tiara Hung5 · Iair Arcavi6,7 Received: 25 April 2020 / Accepted: 13 October 2020 / Published online: 27 October

Mission: HST
ID: paper1725_mission0_HST_MENTION
Text snippet:
1 Introduction Strong galaxy-galaxy lensing produces multiple, distorted images of a background galaxy through the gravitational field of the foreground galaxy. It was first suggested by Zwicky ( 1937a ) for galaxy-clusters, and the first discovery of a galaxy-scale lens was made by Walsh et al. ( 1979 ). The development in optics has led to a rapid increase in the number of strong lensing system discovered. In 2010, the total number of confirmed strong gravitational lensing by galaxies was appr
---


Text #169:
Classification: DATA_INFLUENCED
Mission: HST
ID: paper1728_mission0_HST_DATA_INFLUENCED
Text snippet:
1 Introduction Jupiters moon Io is embedded in a sub-Alfvnic flow of Jupiters corotating magnetospheric plasma, which constantly overtakes the moon and interacts with Ios atmosphere. From this interaction, different wave modes are excited with the Alfvn mode being an especially important mode as it can transport mome

Text snippet:
MNRAS 000, 1–11 (2020) Preprint 20 October 2020 Compiled using MNRAS LATEX style file v3.0 LESSER: A Catalogue of Spectroscopically selected sample of Lyman-α Emitters Lensed By Galaxies Xiaoyue Cao,1,2 Ran Li2,1★ , Yiping Shu,3 Shude Mao,4,2 Jean-Paul Kneib,5,6 Liang Gao2,1 1 School of Astronomy and Space Science, University of Chinese Academy of Sciences, Beijing 100049, China Astronomical Observatories, Chinese Academy of Sciences, 20A Datun Road, Chaoyang District, Beijing 100012, China 3 In
---


Text #383:
Classification: SCIENCE
Mission: HST
ID: paper2809_mission0_HST_SCIENCE
Text snippet:
1 INTRODUCTION Quasar absorption line observations remain one of the most sensitive tools for examining the diffuse gaseous component of the Universe. At low redshifts ( z ≲ 1), such studies are particularly challenging as the gas beyond galaxies is inhomogeneously distributed over a broad temperature–density range. The circumgalactic medium (CGM) is one of the major reservoirs o

In [13]:
#Print number of texts that fell under given parameters
print("Target missions:")
for curr_kobj in all_kobjs:
    print(curr_kobj)
    print("")
print("")
print("Number of valid text entries:")
print(len(dict_texts))

Target missions:
Keyword Object:
Name: Hubble
Keywords: ['Hubble Space Telescope', 'Hubble Telescope', 'Hubble']
Acronyms: ['hst', 'ht']



Number of valid text entries:
500


In [14]:
#Use the Operator instance to train an ML model
start=time.time()
tabby_ML.train_model_ML(dir_model=dir_model, name_model=name_model, do_reuse_run=do_reuse_run,
                        seed_ML=seed_ML, seed_TVT=seed_TVT, filename_json=None, dict_texts=dict_texts,
                        buffer=buffer, fraction_TVT=fraction_TVT, mode_TVT=mode_TVT, do_shuffle=do_shuffle,
                        do_verbose=True, do_verbose_deep=None)

print(f'Time to train the model with run = {time.time()-start}')
#


> Running train_model_ML()!
Loading data from given .json file or dict. of texts...
Text data has been loaded.
Processing text data into modifs...
25 of 500 total texts have been processed...
50 of 500 total texts have been processed...
75 of 500 total texts have been processed...
100 of 500 total texts have been processed...
125 of 500 total texts have been processed...
150 of 500 total texts have been processed...
175 of 500 total texts have been processed...
200 of 500 total texts have been processed...
225 of 500 total texts have been processed...
250 of 500 total texts have been processed...
275 of 500 total texts have been processed...
300 of 500 total texts have been processed...
325 of 500 total texts have been processed...
350 of 500 total texts have been processed...
375 of 500 total texts have been processed...
400 of 500 total texts have been processed...
425 of 500 total texts have been processed...
450 of 500 total texts have been processed...
475 of 500 total texts have

INFO:absl:Using /var/folders/0g/b03r16sd1kl3_j3fml_xwl9h0002td/T/tfhub_modules to cache modules.


Loading testing data...
Found 311 files belonging to 3 classes.
Done loading datasets.
Loading ML model components...
Loaded ML preprocessor: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
Loaded ML encoder: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Done loading ML model components.
Building an empty ML model...
Dropout fraction: 0.2
Number of Dense layers: 3


INFO:absl:using Lamb optimizer


Done building an empty ML model.
Setting up loss, metric, and optimization functions...
# of training steps: 30
# of warmup steps: 3
Type of optimizer and initial lr: lamb, 3e-05
Done compiling loss, metric, and optimization functions.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessor (KerasLayer)      {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),            

INFO:tensorflow:Assets written to: /Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/scratchwork/models/test_run/tfoutput_test_run/assets


INFO:tensorflow:Assets written to: /Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/scratchwork/models/test_run/tfoutput_test_run/assets



Training complete.
New ML model trained and stored in /Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/scratchwork/models/test_run.
Run of train_model_ML() complete!

Time to train the model with run = 211.65752696990967


And with that, we're done training a new ML model!  If run successfully, the model will be saved in the `dir_model` directory.

We can then use the brand new model to classify some new text, like so:

In [15]:
#Set path to the new model output
filepath_model = os.path.join(dir_model, (name_model+".npy"))
fileloc_ML = os.path.join(dir_model, (preset.tfoutput_prefix+name_model))
#Load the new ML model into a new Classifier_ML instance
classifier_ML = bibcat.Classifier_ML(filepath_model=filepath_model, fileloc_ML=fileloc_ML,
                                    do_verbose=True)
#
#Load the instance into a new Operator
tabby_ML = bibcat.Operator(classifier=classifier_ML, mode=mode_modif, keyword_objs=all_kobjs,
                           do_verbose=True, load_check_truematch=True, do_verbose_deep=False)
#

INFO:absl:using Lamb optimizer


Instance of Operator successfully initialized!
Keyword objects:
0: Keyword Object:
Name: Hubble
Keywords: ['Hubble Space Telescope', 'Hubble Telescope', 'Hubble']
Acronyms: ['hst', 'ht']



In [16]:
#Run the classifier for some sample text below
lookup = "HST"
text = "In this study, we present our HST observations of stars in the star-forming region Taurus."
threshold = 0.8
#
#Run the classifier
result = tabby_ML.classify(text=text, lookup=lookup, buffer=0, threshold=threshold,
                            do_raise_innererror=False, do_check_truematch=True)
#

> Running _fetch_keyword_object() for lookup term HST.
Best matching keyword object (keyobj) for keyword HST:
Keyword Object:
Name: Hubble
Keywords: ['Hubble Space Telescope', 'Hubble Telescope', 'Hubble']
Acronyms: ['hst', 'ht']


Preprocessing and extracting modifs from the text...

Running Grammar on the text...
Text has been processed into modifs.
Text has been processed into modif.

Running classify_text for ML classifier:
Class names from model:
['DATA_INFLUENCED', 'MENTION', 'SCIENCE']


Method classify_text for ML classifier complete!
Max verdict: DATA_INFLUENCED



In [17]:
#Print the classifier results
print("Modif: {2}\n\nClassification: {0}\n\nUncertainties per class: {1}\n"
      .format(result["verdict"], result["uncertainty"], result["modif"]))
print("Full classification output:\n{0}".format(result))

Modif: In this study, we present our OBJ observations of stars in the star - forming region Taurus.

Classification: z_lowprob

Uncertainties per class: {'DATA_INFLUENCED': 0.43682823, 'MENTION': 0.32604313, 'SCIENCE': 0.23712872}

Full classification output:
{'verdict': 'z_lowprob', 'scores_comb': None, 'scores_indiv': None, 'uncertainty': {'DATA_INFLUENCED': 0.43682823, 'MENTION': 0.32604313, 'SCIENCE': 0.23712872}, 'modif': 'In this study, we present our OBJ observations of stars in the star - forming region Taurus.'}


---

In [18]:
#Set end marker for this tutorial.
print("This tutorial completed successfully.")

This tutorial completed successfully.
